In [ ]:
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import f1_score

import json
import pandas as pd

from collections import Counter

In [ ]:
FILE_PATH = '../../data/processed/data.json'

with open(FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
X, y = zip(*[(key, recipe['geographical_tags_updated']) for key, recipe in data.items()])

In [ ]:
X_train_index, X_test_index, y_train, y_test = train_test_split(X, y, random_state=44)

In [ ]:
X_train = []
X_test = []
for i in X_train_index:
    X_train.append(data[i]['ingredient_annotations'])
for j in X_test_index:
    X_test.append(data[j]['ingredient_annotations'])

In [ ]:
ingr_binarizer = MultiLabelBinarizer()
ingr_binarizer.fit(X_train)

X_train = ingr_binarizer.transform(X_train)
X_test = ingr_binarizer.transform(X_test)

tags_binarizer = MultiLabelBinarizer()
tags_binarizer.fit(y_train)

y_train = tags_binarizer.transform(y_train)
y_test = tags_binarizer.transform(y_test)

In [ ]:
clf = RidgeClassifier(alpha=1).fit(X_train, y_train)

In [ ]:
f1_score(y_test, clf.predict(X_test), average='macro',zero_division = 0)

In [ ]:
y_test_changed = tags_binarizer.inverse_transform(y_test)
pred_changed = tags_binarizer.inverse_transform(clf.predict(X_test))

In [ ]:
f1 = f1_score(y_test, clf.predict(X_test), average=None,zero_division = 1)

In [ ]:
dict1 = dict()
for i,j in zip(tags_binarizer.classes_, f1):
    dict1[i] = [j]

In [ ]:
li = []
for i in y_test_changed:
    for j in i:
        li.append(j)

In [ ]:
count = Counter(li)

In [ ]:
for i,j in count.items():
    dict1[i].append(j)

In [ ]:
sd = pd.DataFrame.from_dict(dict1,orient='index')
sd.rename(columns = {0:'F1-Score',1:'Count'},inplace = True)

In [ ]:
sd.sort_values('F1-Score', ascending=False)